### Feature impotance with desidion tree

In [5]:
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Загрузка набора данных о винах
data = load_wine()
X = data.data
y = data.target

# Обучение дерева решений и оценка важности признаков
model = DecisionTreeClassifier(random_state=0)
model.fit(X, y)
feature_importances = model.feature_importances_

feature_importances


array([0.01257056, 0.02560169, 0.        , 0.        , 0.03297845,
       0.        , 0.14144668, 0.        , 0.        , 0.03470451,
       0.05818509, 0.31204257, 0.38247045])

In [7]:
# Первоначальная оценка модели дерева решений без выбора признаков
initial_score = np.mean(cross_val_score(DecisionTreeClassifier(random_state=0), X, y, cv=5))

# Оценка модели дерева решений, используя признаки, выбранные деревом решений
selected_X_by_tree = X[:, feature_importances > np.mean(feature_importances)]
score_with_tree_selected_features = np.mean(cross_val_score(DecisionTreeClassifier(random_state=0), selected_X_by_tree, y, cv=5))

print("No features selection: ", initial_score)
print("Desidion tree feature selection: ", score_with_tree_selected_features)


No features selection:  0.8876190476190475
Desidion tree feature selection:  0.8996825396825396


### Алгоритм ADD-DEL (добавление-удаление)

Алгоритм ADD-DEL (добавление-удаление) — это эвристический метод, используемый в машинном обучении и оптимизации для выбора признаков или переменных в модели. Он работает в двух основных этапах:

Этап добавления (ADD): Начиная с пустого набора признаков, алгоритм последовательно добавляет признаки, которые наиболее улучшают производительность модели. На каждом шаге алгоритм оценивает, как добавление каждого неиспользуемого признака повлияет на модель, и выбирает тот, который дает наибольшее улучшение. Этот процесс продолжается до тех пор, пока добавление новых признаков не перестанет улучшать модель или пока не будут учтены все признаки.


Этап удаления (DEL): После того, как добавлены все признаки, алгоритм переходит к этапу удаления. На этом этапе он последовательно удаляет признаки, которые меньше всего влияют на производительность модели или даже ухудшают ее. На каждом шаге алгоритм оценивает влияние удаления каждого признака и удаляет тот, чье удаление наиболее положительно сказывается на модели. Этот процесс продолжается до тех пор, пока удаление признаков не начнет ухудшать модель.
Эти два этапа могут чередоваться, пока не будет найден оптимальный набор признаков. Алгоритм ADD-DEL особенно полезен, когда есть большое количество потенциальных признаков, и нужно определить, какие из них наиболее важны для моделирования.

In [9]:


def add_del_feature_selection_dt(X, y, feature_importances, initial_features=[], threshold=0.01):
    """
    ADD-DEL Feature Selection Algorithm using Decision Tree feature importances
    :param X: Feature set
    :param y: Target variable
    :param feature_importances: Importance of features as determined by a decision tree
    :param initial_features: List of initially selected features (indices)
    :param threshold: Improvement threshold for feature addition/removal
    :return: List of selected feature indices
    """
    selected_features = list(initial_features)
    features = list(range(X.shape[1]))
    improvement = True

    while improvement:
        improvement = False

        # ADD Step
        for feature in set(features) - set(selected_features):
            current_features = selected_features + [feature]
            score = np.mean(cross_val_score(DecisionTreeClassifier(random_state=0), X[:, current_features], y, cv=5))
            if score - threshold > 0.01:  # Use a fixed threshold for improvement
                selected_features.append(feature)
                threshold = score
                improvement = True

        # DEL Step
        for feature in selected_features:
            if feature_importances[feature] < np.mean(feature_importances):  # Consider removing less important features
                current_features = list(selected_features)
                current_features.remove(feature)
                score = np.mean(cross_val_score(DecisionTreeClassifier(random_state=0), X[:, current_features], y, cv=5))
                if score - threshold > 0.01:
                    selected_features.remove(feature)
                    threshold = score
                    improvement = True

    return selected_features

# Выбор признаков с помощью алгоритма ADD-DEL
selected_features = add_del_feature_selection_dt(X, y, feature_importances)

# Обучение и оценка модели дерева решений на выбранных признаках
selected_X = X[:, selected_features]
final_model = DecisionTreeClassifier(random_state=0)
final_score = np.mean(cross_val_score(final_model, selected_X, y, cv=5))

print("Features: ", selected_features)
print("ADD-DEL selection score: ", final_score)


Features:  [0, 1, 3, 4, 5, 6, 9, 10]
ADD-DEL selection score:  0.961111111111111


### Feature impotance with permutation

In [11]:
from sklearn.inspection import permutation_importance

# Обучение модели дерева решений на всех признаках
model.fit(X, y)

# Вычисление важности признаков с помощью перестановок
perm_importance = permutation_importance(model, X, y, n_repeats=100, random_state=0)

perm_importance.importances_mean

array([0.02275281, 0.02342697, 0.        , 0.        , 0.01848315,
       0.        , 0.39842697, 0.        , 0.        , 0.02702247,
       0.05747191, 0.07539326, 0.13674157])

Вот ключевые выводы из этих результатов:

Некоторые признаки, такие как 6-й (средняя важность около 0.408), 12-й (0.133), 11-й (0.075), и 10-й (0.058), имеют значительное влияние на производительность модели. Это указывает на их высокую важность для принятия решений.
Другие признаки имеют гораздо меньшую важность, а некоторые из них, как видно, не оказывают значительного влияния на производительность модели (например, признаки 2, 3, 5, 7, 8).